In [23]:
import matplotlib.pyplot as plt
import os
import pandas as pd

In [24]:
def compute_diff_counts(df, metric_name):
    """ 
    Computes the metric_name score difference between transported target \
        score and target score for df if the target score does not exceed the source score
    """
    
    source_scores = df["source_"+metric_name]
    target_scores = df["target_"+metric_name]
    trans_target_scores = df["trans_target_"+metric_name]
    diff_scores = []
    for source_score, target_score, trans_target_score in zip(source_scores, target_scores, trans_target_scores):
        if source_score > target_score:
            diff_scores.append(trans_target_score-target_score)
    return diff_scores


In [15]:
score_dir = "/home/wanxinli/EHR-OT/outputs/mimic/"

f1_diff_counts = []
recall_diff_counts = []
precision_diff_counts = []

f1_degrade_code = []
precision_degrade_code = []
recall_degrade_code = []
code_count = 0

for file in os.listdir(score_dir):
    if file.endswith("_score.csv") and 'exp3_' in file and 'TCA' not in file:
        code_count += 1
        score_df = pd.read_csv(os.path.join(score_dir, file), index_col=None, header=0)

        f1_diffs = compute_diff_counts(score_df, 'recall')
        f1_improve_count = [x>0 for x in f1_diffs].count(True)

        if f1_improve_count <= len(f1_diffs)/2:
            f1_degrade_code.append(file.split("_")[1])

        precision_diffs = compute_diff_counts(score_df, 'precision')
        precision_improve_count = [x>0 for x in precision_diffs].count(True)

        if precision_improve_count <= len(precision_diffs)/2:
            precision_degrade_code.append(file.split("_")[1])
        
        recall_diffs = compute_diff_counts(score_df, 'recall')
        recall_improve_count = [x>0 for x in recall_diffs].count(True)

        if recall_improve_count <= len(recall_diffs)/2:
            recall_degrade_code.append(file.split("_")[1])


In [16]:
print("f1 improve code percent is:", 1-len(f1_degrade_code)/code_count)
print("precision improve code percent is:", 1-len(precision_degrade_code)/code_count)
print("recall improve code percent is:", 1-len(recall_degrade_code)/code_count)

f1 improve code percent is: 0.6625514403292181
precision improve code percent is: 0.51440329218107
recall improve code percent is: 0.6625514403292181


In [43]:
# compare TCA versus EHR-OT
score_dir = "/home/wanxinli/EHR-OT/outputs/mimic/"

f1_diff_counts = []
TCA_diff_counts = []

f1_degrade_code = []
TCA_f1_degrade_code = []
code_count = 0

for file in os.listdir(score_dir):
    if file.endswith("TCA_score.csv") and 'exp3_' in file:
        code_count += 1
        label_code = file.split('_')[1]

        score_path = os.path.join(score_dir, "exp3_"+label_code+"_score.csv")
        score_df = pd.read_csv(score_path, index_col=None, header=0)

        f1_diffs = compute_diff_counts(score_df, 'recall')
        f1_improve_count = [x>0 for x in f1_diffs].count(True)

        if f1_improve_count <= len(f1_diffs)/2:
            f1_degrade_code.append(label_code)

        TCA_score_path = os.path.join(score_dir, file)
        TCA_score_df = pd.read_csv(TCA_score_path, index_col=None, header=0)

        TCA_f1_diffs = compute_diff_counts(TCA_score_df, 'recall')
        TCA_f1_improve_count = [x>0 for x in TCA_f1_diffs].count(True)

        if TCA_f1_improve_count <= len(TCA_f1_diffs)/2:
            TCA_f1_degrade_code.append(label_code)


In [44]:
print(len(TCA_f1_degrade_code), len(TCA_f1_degrade_code)/code_count)

148 0.6090534979423868


In [45]:
print(len(f1_degrade_code), len(f1_degrade_code)/code_count)

82 0.3374485596707819


In [28]:
print(code_count)

243
